In [1]:
#importing used libraries
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from ipywidgets import interact, fixed
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.utils.multiclass import unique_labels
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pandas as pd
import json
import glob
import os
from sklearn.model_selection import cross_val_score
import category_encoders as ce
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV
from xgboost import plot_importance
import eli5
from eli5.sklearn import PermutationImportance
from pdpbox.pdp import pdp_interact, pdp_interact_plot
import shap

pd.set_option("display.max_rows", 101)

In [2]:
path = os.getcwd()
files = [f for f in glob.glob(path + "*/**/*.json", recursive=True)]

def json_to_df(json_path):
    with open(json_path, 'r') as json_data:
        data = json.load(json_data)
    data = pd.DataFrame.from_dict(data, orient='index')
    data = data[0]
    mapid = data['id']
    match_name = data['NameWithoutExtension']
    map_name = data['map_name']
    team1_series_score = data['score_team1']
    team2_series_score = data['score_team2']
    team1_half1_score = data['score_half1_team1']
    team2_half1_score = data['score_half1_team2']
    team2_half2_score = data['score_half2_team2']
    team2_half1_score = data['score_half2_team1']
    team1_name = pd.DataFrame.from_dict(data['team_ct'], orient='index')[0][0]
    team2_name = pd.DataFrame.from_dict(data['team_t'], orient='index')[0][0]


    team_ct_df = pd.DataFrame.from_dict(data['team_ct'], orient='index')




    ct_start_money_df = pd.DataFrame()
    ct_equipment_value_df = pd.DataFrame()
    ct_money_earned_df = pd.DataFrame()
    ct_time_death_rounds = pd.DataFrame()

    for i in range(0,5):
        ct_start_money_df[i] = pd.DataFrame.from_dict(pd.DataFrame(team_ct_df[0][4])['start_money_rounds'][i], orient='index')[0]
        ct_equipment_value_df[i] = pd.DataFrame.from_dict(pd.DataFrame(team_ct_df[0][4])['equipement_value_rounds'][i], orient='index')[0]
        ct_money_earned_df[i] = pd.DataFrame.from_dict(pd.DataFrame(team_ct_df[0][4])['rounds_money_earned'][i], orient='index')[0]
        ct_time_death_rounds[i] = pd.DataFrame.from_dict(pd.DataFrame(team_ct_df[0][4])['time_death_rounds'][i], orient='index')[0]


    ct_start_money_df['Total'] = ct_start_money_df.sum(axis=1)
    ct_equipment_value_df['Total'] = ct_equipment_value_df.sum(axis=1)
    ct_money_earned_df['ct_earnedmoney_total'] = ct_money_earned_df.sum(axis=1)

    ct_start_money_df.index = ct_start_money_df.index.rename('round')
    ct_start_money_df = ct_start_money_df.reset_index()

    ct_equipment_value_df.index = ct_equipment_value_df.index.rename('round')
    ct_equipment_value_df = ct_equipment_value_df.reset_index()

    ct_money_earned_df.index = ct_money_earned_df.index.rename('round')
    ct_money_earned_df = ct_money_earned_df.reset_index()

    ct_time_death_rounds['ct_survived'] = ct_time_death_rounds.isin(['0.0']).sum(axis=1)
    ct_time_death_rounds.index = ct_time_death_rounds.index.rename('round')
    ct_time_death_rounds = ct_time_death_rounds.reset_index()
    ct_time_death_rounds['round'] = ct_time_death_rounds['round'].astype(int) 
    


    team_t_df = pd.DataFrame.from_dict(data['team_t'], orient='index')
    t_start_money_df = pd.DataFrame()
    t_equipment_value_df = pd.DataFrame()
    t_money_earned_df = pd.DataFrame()
    t_time_death_rounds = pd.DataFrame()

    for i in range(0,5):
        t_start_money_df[i] = pd.DataFrame.from_dict(pd.DataFrame(team_t_df[0][4])['start_money_rounds'][i], orient='index')[0]
        t_equipment_value_df[i] = pd.DataFrame.from_dict(pd.DataFrame(team_t_df[0][4])['equipement_value_rounds'][i], orient='index')[0]
        t_money_earned_df[i] = pd.DataFrame.from_dict(pd.DataFrame(team_t_df[0][4])['rounds_money_earned'][i], orient='index')[0]
        t_time_death_rounds[i] = pd.DataFrame.from_dict(pd.DataFrame(team_t_df[0][4])['time_death_rounds'][i], orient='index')[0]


    t_start_money_df['Total'] = t_start_money_df.sum(axis=1)
    t_equipment_value_df['Total'] = t_equipment_value_df.sum(axis=1)
    t_money_earned_df['t_earnedmoney_total'] = t_money_earned_df.sum(axis=1)

    t_start_money_df.index = t_start_money_df.index.rename('round')
    t_start_money_df = t_start_money_df.reset_index()

    t_equipment_value_df.index = t_equipment_value_df.index.rename('round')
    t_equipment_value_df = t_equipment_value_df.reset_index()

    t_money_earned_df.index = t_money_earned_df.index.rename('round')
    t_money_earned_df = t_money_earned_df.reset_index()

    t_time_death_rounds['t_survived'] = t_time_death_rounds.isin(['0.0']).sum(axis=1)
    t_time_death_rounds.index = t_time_death_rounds.index.rename('round')
    t_time_death_rounds = t_time_death_rounds.reset_index()
    t_time_death_rounds['round'] = t_time_death_rounds['round'].astype(int) 
    

    
    allrounds = data.loc['rounds']
    allrounds = pd.DataFrame(allrounds)
    
    roundsdf = pd.DataFrame()
    roundsdf['round'] = allrounds['number']
    roundsdf['winner_side'] = allrounds['winner_side']
    roundsdf['winner_name'] = allrounds['winner_name']
    roundsdf['team_t_name'] = allrounds['team_t_name']
    roundsdf['team_ct_name'] = allrounds['team_ct_name']
    roundsdf['equipment_value_team_t'] = allrounds['equipement_value_team_t']
    roundsdf['equipment_value_team_ct'] = allrounds['equipement_value_team_ct']
    roundsdf['start_money_team_t'] = allrounds['start_money_team_t']
    roundsdf['start_money_team_ct'] = allrounds['start_money_team_ct']
    roundsdf['bomb_defused_count'] = allrounds ['bomb_defused_count']
    roundsdf['bomb_exploded_count'] = allrounds['bomb_exploded_count'] 
    roundsdf['bomb_planted_count'] = allrounds['bomb_planted_count'] 
    roundsdf['duration'] = allrounds['duration']
    roundsdf['type'] = allrounds['type']
    
    roundsdf = pd.merge(roundsdf,ct_time_death_rounds[['round','ct_survived']],on='round',how='left')
    roundsdf = pd.merge(roundsdf,t_time_death_rounds[['round','t_survived']],on='round',how='left')
    
    ct_money_earned_df['round'] = ct_money_earned_df['round'].astype(int)
    ct_money_earned_df[['round','ct_earnedmoney_total']]
    t_money_earned_df['round'] = t_money_earned_df['round'].astype(int)
    t_money_earned_df[['round','t_earnedmoney_total']]
    
        
    roundsdf['winner_side'] = roundsdf['winner_side'].replace('CT',0)
    roundsdf['winner_side'] = roundsdf['winner_side'].replace('T',1)
    
    roundsdf['mapid'] = mapid
    roundsdf['match_name'] = match_name
    roundsdf['map_name'] = map_name
    
    roundsdf['team1_series_score'] = team1_series_score
    roundsdf['team2_series_score']= team2_series_score
       
    return roundsdf



In [3]:
path = os.getcwd()
train_path = str(path)+"/train_majors/"
all_files = [f for f in glob.glob(train_path + "**/**.csv", recursive=True)]

In [4]:
for f in files:
    filepath = f.split('.dem.')[0]
    df_json = json_to_df(f)
    df_json.to_csv(str(filepath)+".csv")
    print(len(df_json))

In [5]:
# all_files = [f for f in glob.glob(path + "*/**/*.csv", recursive=True)]
# for f in all_files:
#     os.remove(f)

In [6]:
path = os.getcwd()
train_path = str(path)+"/train_majors/"
train_files = [f for f in glob.glob(train_path + "**/**.csv", recursive=True)]

li = []

for filename in train_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

traindf = pd.concat(li, axis=0, ignore_index=True)
traindf = traindf.drop('Unnamed: 0',axis=1)
traindf = traindf[traindf['winner_side'] !='SPEC']

ValueError: No objects to concatenate

In [ ]:
path = os.getcwd()
train_path = str(path)+"/test_major/"
train_files = [f for f in glob.glob(train_path + "**/**.csv", recursive=True)]

li = []

for filename in train_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

testdf = pd.concat(li, axis=0, ignore_index=True)
testdf = testdf.drop('Unnamed: 0',axis=1)

In [ ]:
len(traindf)

In [ ]:
len(testdf)

In [ ]:
def format_csvs(df):
    #rearrange the columns
    df_cols = ['round', 'equipment_value_team_t', 'equipment_value_team_ct','start_money_team_t',
        'start_money_team_ct','bomb_defused_count','bomb_exploded_count','bomb_planted_count','duration',
               'type', 'ct_survived', 't_survived', 'mapid', 'match_name', 'map_name',
            'team1_series_score', 'team2_series_score', 'winner_name', 'team_t_name', 'team_ct_name',
            'winner_side']
    df = df[df_cols]
    #copying the dataframe to avoid warning messages
    df = df.copy()
    
    
    #initializing the series columns
    df['team1_current_score'] = 0 * len(df)
    df['team2_current_score'] = 0 * len(df)
    #grabbing the amount of rows in the dataframe
    rows, _ = df.shape
    #dummyvalue for first entry in the dataframe
    lastmatch = 'newdf'
    
    for arow in range(0,rows):
        #assign the winner of the round and matchid
        roundwinner = df.iloc[arow]['winner_name']
        roundmapid = df.iloc[arow]['mapid']
        if roundmapid != lastmatch:
            #if the matchid does not match the previous row's matchid
            #this is a new series
            
            #assigning the team names
            team2_name = df.iloc[arow]['team_t_name']
            team1_name = df.iloc[arow]['team_ct_name']          
            if roundwinner == team1_name:
                df.at[arow,'team1_current_score'] = 1
                lastmatch = df.iloc[arow]['mapid']
            else:
                df.at[arow,'team2_current_score']= 1
                lastmatch = df.iloc[arow]['mapid']
        else:
            #if it is the same match, pulling the last row allows us to
            #append to the previous series score
            lastrow = df.iloc[(arow-1)]
            old_t1_score = lastrow['team1_current_score'].astype(int)
            old_t2_score = lastrow['team2_current_score'].astype(int)
            
            if df.iloc[arow]['winner_name'] == team1_name:
                lastmatch = df.iloc[arow]['mapid']
                df.at[arow,'team1_current_score']=(old_t1_score+1)
                df.at[arow,'team2_current_score']=(old_t2_score)
            else:
                lastmatch = df.iloc[arow]['mapid']
                df.at[arow,'team1_current_score']=(old_t1_score)
                df.at[arow,'team2_current_score']=(old_t2_score+1)
                
    #measuring the net surplus each team has in equipment value at the start of the round
    
    
    df['equipment_value_team_t_surplus'] = df['equipment_value_team_t']-df['equipment_value_team_ct']
    df['equipment_value_team_ct_surplus'] = df['equipment_value_team_ct']-df['equipment_value_team_t']
    df['start_money_team_t_surplus'] = df['start_money_team_t'] - df['start_money_team_ct']
    df['start_money_team_ct_surplus'] = df['start_money_team_ct'] - df['start_money_team_t']
 
    ##this creates the value of a team's equipment last round
    df['equipment_value_team_t_surplus_LR'] = df['equipment_value_team_t_surplus'].shift(1)
    df['equipment_value_team_ct_surplus_LR'] = df['equipment_value_team_ct_surplus'].shift(1)
    df['equipment_value_team_ct_surplus_LR'][df['round']==1] = np.NaN
    df['equipment_value_team_t_surplus_LR'][df['round']==1] = np.NaN
    df['equipment_value_team_ct_surplus_LR'][df['round']==16] = np.NaN
    df['equipment_value_team_t_surplus_LR'][df['round']==16] = np.NaN
    df['equipment_value_team_ct_surplus_LR'][df['round']==31] = np.NaN
    df['equipment_value_team_t_surplus_LR'][df['round']==31] = np.NaN
    df['equipment_value_team_ct_surplus_LR'][df['round']==46] = np.NaN
    df['equipment_value_team_t_surplus_LR'][df['round']==46] = np.NaN
    
    
    ##this creates the value of economy last round
    df['start_money_team_t_surplus_LR'] = df['start_money_team_t_surplus'].shift(1)
    df['start_money_team_ct_surplus_LR'] = df['start_money_team_ct_surplus'].shift(1)
    df['start_money_team_ct_surplus_LR'][df['round']==1] = np.NaN
    df['start_money_team_t_surplus_LR'][df['round']==1] = np.NaN
    df['start_money_team_ct_surplus_LR'][df['round']==16] = np.NaN
    df['start_money_team_t_surplus_LR'][df['round']==16] = np.NaN
    df['start_money_team_ct_surplus_LR'][df['round']==31] = np.NaN
    df['start_money_team_t_surplus_LR'][df['round']==31] = np.NaN
    df['start_money_team_ct_surplus_LR'][df['round']==46] = np.NaN
    df['start_money_team_t_surplus_LR'][df['round']==46] = np.NaN    
        
    ##this creates the length of last round as a feature
    df['lastround_duration'] = df['duration'].shift(1)
    df['lastround_duration'][df['round']==1] = np.NaN
    
    
    
    #this creates number survived last round
    df['ct_survived_LR'] = df['ct_survived'].shift(1)
    df['t_survived_LR'] = df['t_survived'].shift(1)
    df['ct_survived_LR'][df['round']==1] = np.NaN
    df['t_survived_LR'][df['round']==16] = np.NaN
    df['ct_survived_LR'][df['round']==1] = np.NaN
    df['t_survived_LR'][df['round']==16] = np.NaN
    df['ct_survived_LR'][df['round']==30] = np.NaN
    df['t_survived_LR'][df['round']==30] = np.NaN
    df['ct_survived_LR'][df['round']==45] = np.NaN
    df['t_survived_LR'][df['round']==45] = np.NaN
    
    df['team1_current_score_LR'] = df['team1_current_score'].shift(1)
    df['team2_current_score_LR'] = df['team2_current_score'].shift(1)
    df['team1_current_score_LR'][df['round']==1] = np.NaN
    df['team2_current_score_LR'][df['round']==1] = np.NaN
    
    
    
    
    
    ##this creates the value of economy last round
    df['t_win_LR'] = df['winner_side'].shift(1)
    df['t_win_LR'][df['round']==1] = np.NaN
    df['t_win_LR']=df['t_win_LR'].astype(float)
    
     
    map_name = df['map_name']
    map_name = map_name.replace('workshop/125438669/de_inferno','de_inferno')
    map_name = map_name.replace('workshop/125438255/de_dust2','de_dust2')
    map_name = map_name.replace('workshop/152508932/de_mirage ','de_mirage')
    map_name = map_name.replace('workshop/125439125/de_nuke','de_nuke')
    map_name = map_name.replace('workshop/125438372/de_train','de_train')
    map_name = map_name.replace('workshop/152508932/de_mirage','de_mirage')
    df['map_name'] = map_name
    df = df[df['map_name']!='de_vertigo']
    df['eqipment_value_ct_swing'] = df['equipment_value_team_ct_surplus'] - df['start_money_team_ct_surplus_LR']
    df['eqipment_value_t_swing'] = df['equipment_value_team_t_surplus'] - df['start_money_team_t_surplus_LR']
    df = df[~df['winner_side'].isna()]
    df['t_win_LR'] = df['t_win_LR'].astype(float)
    df['winner_side'] = df['winner_side'].astype(float)
    
    rows, _ = df.shape
    for arow in range(0,rows):
        #assign the winner of the round and matchid
        roundnumber = df.iloc[arow]['round']
        side = roundnumber//16
        team1score = df.iloc[arow]['team1_current_score_LR']
        team2score = df.iloc[arow]['team2_current_score_LR']
        if side == 0:
            df.at[arow,'ct_score_LR'] = team1score
            df.at[arow,'t_score_LR'] = team2score
        elif side == 1:
            df.at[arow,'ct_score_LR'] = team2score
            df.at[arow,'t_score_LR'] = team1score
        elif side == 2:
            df.at[arow,'ct_score_LR'] = team1score
            df.at[arow,'t_score_LR'] = team2score
        elif side == 3:
            df.at[arow,'ct_score_LR'] = team2score
            df.at[arow,'t_score_LR'] = team1score
        else:
            df.at[arow,'ct_score_LR'] = team1score
            df.at[arow,'t_score_LR'] = team2score        
    df['ct_score_LR'][df['round']==1] = np.NaN
    df['ct_score_LR'][df['round']==1] = np.NaN
    df = df[~df['winner_side'].isna()]
    return df

In [ ]:
# rows, _ = df.shape
# for arow in range(0,rows):
#     #assign the winner of the round and matchid
#     roundnumber = df.iloc[arow]['round']
#     side = roundnumber//16
#     team1score = df.iloc[arow]['team1_current_score_LR']
#     team2score = df.iloc[arow]['team2_current_score_LR']
#     if side == 0:
#         df['ct_score_LR'] = team1score
#         df['t_score_LR'] = team2score
#     elif side == 1:
#         df['ct_score_LR'] = team2score
#         df['t_score_LR'] = team1score
#     elif side == 2:
#         df['ct_score_LR'] = team1score
#         df['t_score_LR'] = team2score
#     elif side == 3:
#         df['ct_score_LR'] = team2score
#         df['t_score_LR'] = team1score
#     elif side == 4:
#         df['ct_score_LR'] = team1score
#         df['t_score_LR'] = team2score        

In [ ]:
train_df = format_csvs(traindf)

In [ ]:
test_df = format_csvs(testdf)

In [ ]:
train_df.columns

In [ ]:
features = ['round','equipment_value_team_t','equipment_value_team_ct','ct_score_LR','t_score_LR','map_name']
target = 'winner_side'

In [ ]:
# train_df = train_df.fillna(-99)
# test_df = test_df.fillna(-99)

In [ ]:
X_train = train_df[features]
y_train = train_df[target]

X_test = test_df[features]
y_test = test_df[target]

#encoding the one None as CT Win
#drop match or fix later
#y_all = y_all.replace('None',0)

In [ ]:
#splitting the train set into an 80/20 train/validation set
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.25,random_state=1337)

In [ ]:
# X_train, X_val, y_train, y_val = train_test_split(
#     X_train, y_train, test_size=0.375,random_state=1337)

In [ ]:
encodin = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True), 
    XGBClassifier(max_depth=3, 
                  n_estimators=200,
                  n_jobs=-1,
                  learning_rate=0.02,
                  random_state=1337,
                  eval_set=eval_set,
                  eval_metric='auc',
                  early_stopping_rounds=25)
)


In [ ]:
eval_set = [(X_train_transformed,y_train),(X_val_transformed,y_val)]

params = {
        'min_child_weight': [1,2,3,5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5,6],
        'n_estimators': [100,150,200,300,500],
        #'learning_rate': np.random.uniform(0.01,0.2)
        }

k = 3
param_comb = 500

transformers = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True)
)

X_train_transformed = transformers.fit_transform(X_train)
X_val_transformed = transformers.transform(X_val)
X_test_transformed = transformers.transform(X_test)


model = XGBClassifier(eval_set=eval_set,eval_metric='auc',early_stopping_rounds=50)



random_search = RandomizedSearchCV(model, 
                                   param_distributions=params, 
                                   n_iter=param_comb, 
                                   scoring='roc_auc', 
                                   n_jobs=-1, 
                                   cv=k, 
                                   verbose=3,
                                   random_state=1337 )
random_search.fit(X_train_transformed, y_train)

In [ ]:
print('Best hyperparameters', random_search.best_params_)
print('Cross-validation ROC AUC', random_search.best_score_)

In [ ]:
X_train_transformed = transformers.fit_transform(X_train)
X_val_transformed = transformers.transform(X_val)
X_test_transformed = transformers.transform(X_test)

pipelinebest = random_search.best_estimator_



In [ ]:
#we need the proba function (probability of being classified as a 1 to do aucroc analysis)
#this is why we use the .predict_proba and not just the class outcome in .predict
#we select all the values of the second column (positive likelyhook) for the auc curve

y_pred_proba = pipelinebest.predict_proba(X_test_transformed)[:, 1]
y_val_pred_proba = pipelinebest.predict(X_val_transformed)


print('Validation Accuracy', accuracy_score(y_val_pred_proba, y_val))
print('\nTest ROC AUC:', roc_auc_score(y_test, y_pred_proba))


In [ ]:
# Get feature importances
xgb = pipeline.named_steps['xgbclassifier']
ohe = pipeline.named_steps['onehotencoder']


importances = pd.Series(xgb.feature_importances_, ohe.feature_names)

# Plot feature importances
%matplotlib inline
import matplotlib.pyplot as plt

n = 20
plt.figure(figsize=(10,n/2))
plt.title(f'Top {n} features')
importances.sort_values()[-n:].plot.barh(color='grey');

In [ ]:
# for column in X_test.columns:
#     # Fit without column
#     pipeline = make_pipeline(
#         ce.OneHotEncoder(use_cat_names=True), 
#         XGBClassifier(max_depth=4, n_estimators=200,n_jobs=-1, random_state=1337,eval_set=eval_set,eval_metric='auc',early_stopping_rounds=100)
#     )
#     pipeline.fit(X_train.drop(columns=column), y_train)
#     score_without = pipeline.score(X_val.drop(columns=column), y_val)
#     print(f'Validation Accuracy without {column}: {score_without}')

#     # Fit with column
#     pipeline = make_pipeline(
#         ce.OneHotEncoder(use_cat_names=True), 
#         XGBClassifier(max_depth=4, n_estimators=200,n_jobs=-1, random_state=1337,eval_set=eval_set,eval_metric='auc',early_stopping_rounds=100)
#     )
#     pipeline.fit(X_train, y_train)
#     score_with = pipeline.score(X_val, y_val)
#     print(f'Validation Accuracy with {column}: {score_with}')

#     # Compare the error with & without column
#     print(f'Drop-Column Importance for {column}: {score_with - score_without}')
#     print('\n')
#     print('\n')

In [ ]:
model = XGBClassifier(max_depth=3, 
                  n_estimators=200,
                  n_jobs=-1,
                  learning_rate=0.02,
                  random_state=1337,
                  eval_set=eval_set,
                  eval_metric='auc',
                  early_stopping_rounds=25)

transformers = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True)
)

X_train_transformed = transformers.fit_transform(X_train)
X_val_transformed = transformers.transform(X_val)
X_test_transformed = transformers.transform(X_test)


model.fit(X_train_transformed,y_train)

permuter = PermutationImportance(
    model,
    scoring='accuracy', 
    n_iter=10, 
    random_state=1337
)



#oldmodel
#model = XGBClassifier(max_depth=4, n_estimators=100,n_jobs=-1, random_state=1337,eval_set=eval_set,eval_metric='auc',early_stopping_rounds=50)



#dropping pistol rounds for PDP/PI analysis
PI_X_train_transformed = X_train_transformed.fillna(0)
PI_X_val_transformed = X_val_transformed.fillna(0)
PI_X_test_transformed = X_test_transformed.fillna(0)


permuter.fit(PI_X_val_transformed, y_val)
feature_names = PI_X_val_transformed.columns.tolist()

eli5.show_weights(
    permuter, 
    top=None, # show permutation importances for all features
    feature_names=feature_names
)

In [ ]:


features = ['equipment_value_team_t','equipment_value_team_ct']

interaction = pdp_interact(
    model=model, 
    dataset=PI_X_val_transformed, 
    model_features=PI_X_val_transformed.columns, 
    features=features
)

pdp_interact_plot(interaction, plot_type='grid', feature_names=features);

In [ ]:
from pdpbox.pdp import pdp_interact, pdp_interact_plot

features = ['t_score_LR','ct_score_LR']

interaction = pdp_interact(
    model=model, 
    dataset=PI_X_val_transformed, 
    model_features=PI_X_val_transformed.columns, 
    features=features
)

pdp_interact_plot(interaction, plot_type='grid', feature_names=features);

In [ ]:
X_test_transformed.columns

In [ ]:
maps = ['map_name_de_nuke', 'map_name_de_dust2','map_name_de_mirage','map_name_de_overpass','map_name_de_inferno','map_name_de_cbble','map_name_de_cache','map_name_de_train']

In [ ]:
for amap in maps:

    features = ['equipment_value_team_t',amap]

    interaction = pdp_interact(
        model=model, 
        dataset=PI_X_test_transformed, 
        model_features=PI_X_test_transformed.columns, 
        features=features
    )

    pdp_interact_plot(interaction, plot_type='grid', feature_names=features);

In [ ]:
eval_set = [(X_train,y_train),(X_val,y_val)]

pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True), 
    XGBClassifier(max_depth=4, n_estimators=100,n_jobs=-1, random_state=1337,eval_set=eval_set,eval_metric='auc',early_stopping_rounds=50)
)

pipeline.fit(X_train,y_train);

In [ ]:
model = pipeline[-1]       # get the last step in the pipeline
processor = pipeline[:-1]  # get all steps except for the last step

In [ ]:
row = X_test.iloc[[10]]

In [ ]:
test_df['winner_side'].mean()

In [ ]:

explainer = shap.TreeExplainer(model)
row_processed = processor.transform(row)
shap_values = explainer.shap_values(row_processed)

shap.initjs()
print("Expected value is:"+str(explainer.expected_value)),
shap.force_plot(
    base_value=explainer.expected_value, 
    shap_values=shap_values, 
    features=row_processed
)

In [ ]:
test_df['y_pred_proba'] = model.predict_proba(X_test_transformed)[:, 1]

In [ ]:
df_evaluate = pd.DataFrame({
    'index': test_df.index, 
    'pred_proba': test_df['y_pred_proba'], 
    't_win': test_df['winner_side']
})


In [ ]:
X_test_index = X_test.reset_index()

In [ ]:
df_evaluate

In [ ]:
df_evaluate = df_evaluate.merge(
    X_test_index,
    how='left',
    on='index')

In [ ]:
y_test = y_test.reset_index().drop('index',axis=1)

In [ ]:
y_test = y_test['winner_side']

In [ ]:
df_evaluate

In [ ]:
t_win = y_test == 1
ct_win = ~t_win
right = (t_win) == (df_evaluate['pred_proba'] > 0.50)
wrong = ~right



In [ ]:
df_evaluate[t_win & right].sort_values(by='pred_proba',ascending=False)

In [ ]:
df_evaluate[t_win & wrong].sort_values(by='pred_proba',ascending=False)

In [ ]:
df_evaluate[ct_win & right].sort_values(by='pred_proba',ascending=False)

In [ ]:
df_evaluate[ct_win & wrong].sort_values(by='pred_proba',ascending=False)

In [ ]:
from joblib import dump
dump(pipeline, 'pipeline.joblib')

In [ ]:
import pkg_resources
import types
def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            # Split ensures you get root package, 
            # not just imported function
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):
            name = val.__module__.split(".")[0]

        # Some packages are weird and have different
        # imported names vs. system/pip names. Unfortunately,
        # there is no systematic way to get pip names from
        # a package's imported name. You'll have to had
        # exceptions to this list manually!
        poorly_named_packages = {
            "PIL": "Pillow",
            "sklearn": "scikit-learn"
        }
        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]

        yield name
imports = list(set(get_imports()))

# The only way I found to get the version of the root package
# from only the name of the package is to cross-check the names 
# of installed packages vs. imported packages
requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))

for r in requirements:
    print("{}=={}".format(*r))